<a href="https://colab.research.google.com/github/Dharani1999/Word-embedding-techniques/blob/master/Data_pre_process_n_tokenization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from gensim import corpora, models, similarities
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn
from sklearn.model_selection import train_test_split

from gensim.models import TfidfModel
import time
import re
import string
import nltk
nltk.download('punkt')
from nltk import word_tokenize,sent_tokenize
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Dropout
from wordcloud import WordCloud, STOPWORDS
from gensim.parsing.preprocessing import stem_text

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
movies1 = pd.read_csv('/content/drive/My Drive/Movielensdata/ml25m/movies.csv')
tags1 = pd.read_csv('/content/drive/My Drive/Movielensdata/ml25m/tags.csv')
data0 = pd.merge(movies1,tags1)
data0.drop('timestamp',axis=1, inplace=True)
data0.drop('userId',axis=1, inplace=True)

In [3]:
tags1.head()

,userId,movieId,tag,timestamp
0,3,260,classic,1439472355
1,3,260,sci-fi,1439472256
2,4,1732,dark comedy,1573943598
3,4,1732,great dialogue,1573943604
4,4,7569,so bad it's good,1573943455


In [4]:
data0.head()

,movieId,title,genres,tag
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,Owned
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,imdb top 250
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,Pixar
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,Pixar
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,time travel


In [5]:
#shuffling
data1 = data0.sample(frac=1)
data1.head(10)

,movieId,title,genres,tag
15917,163,Desperado (1995),Action|Romance|Western,crime
362088,5617,Secretary (2002),Comedy|Drama|Romance,workplace
498072,31658,Howl's Moving Castle (Hauru no ugoku shiro) (2...,Adventure|Animation|Fantasy|Romance,whimsical
187506,2076,Blue Velvet (1986),Drama|Mystery|Thriller,surreal
916484,125263,The Burglar (1957),Crime|Drama,black and white
416325,7063,"Aguirre: The Wrath of God (Aguirre, der Zorn G...",Adventure|Drama,adventure
352886,5349,Spider-Man (2002),Action|Adventure|Sci-Fi|Thriller,Kirsten Dunst
529642,41566,"Chronicles of Narnia: The Lion, the Witch and ...",Adventure|Children|Fantasy,talking animals
88643,994,Big Night (1996),Comedy|Drama,Isabella Rossellini
290342,3916,Remember the Titans (2000),Drama,racism


In [6]:
data = data1.iloc[0:2000,:]

In [7]:
dev, val = train_test_split(data, test_size=0.5, random_state=0)

In [8]:
dev.isnull().any()

movieId    False
title      False
genres     False
tag        False
dtype: bool

In [9]:
val.isnull().any()

movieId    False
title      False
genres     False
tag        False
dtype: bool

In [10]:
#stoplist = stopwords.words('english')
stoplist=STOPWORDS

def pre_processing(d):
    if (d is None):
        return []
    d = d.lower()
    d = re.sub('[^a-z ]',' ',d)
    #d = re.sub(r'\b\w{1,2}\b','',d)
    #pattern = re.compile(r'\b(' + r'|'.join(stoplist) + r')\b\s*')
    #d = pattern.sub('', d)
    d = re.sub("  *"," ",d)
    return (d.strip())

In [11]:
#def preprocess(text):
 #   clean_data = []
  #  new_text = re.sub('<.*?>', '', text)
   # new_text = re.sub(r'[^\w\s]', '', new_text)
    #new_text = re.sub(r'\d+','',new_text)
    #new_text = new_text.lower()
    #if new_text != '':
     #   clean_data.append(new_text)
    #return clean_data

def Clean_names(movie_name): 
    # Search for opening bracket in the name followed by 
    # any characters repeated any number of times 
    if re.search('\(.*', movie_name): 
        # Extract the position of beginning of pattern 
        pos = re.search('\(.*', movie_name).start() 
        # return the cleaned name 
        return movie_name[:pos] 
    else: 
        # if clean up needed return the same name 
        return movie_name 

In [12]:
dev['title'] = dev['title'].apply(Clean_names)
#print(dev['title'])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [13]:
dev.head()

,movieId,title,genres,tag
498480,31660,Steamboy,Action|Animation|Drama|Sci-Fi,steampunk
702571,77629,Reindeerspotting - Pako joulumaasta,Documentary,addiction
947629,138940,Duska,Drama,loneliness
1072230,187613,The Kissing Booth,Comedy|Romance,bff
7563,47,Seven,Mystery|Thriller,psychology


In [14]:
dev['title'] = dev['title'].apply(pre_processing)
dev['genres'] = dev['genres'].apply(pre_processing)
dev['tag'] = dev['tag'].apply(pre_processing)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation:

In [15]:
dev.head()

,movieId,title,genres,tag
498480,31660,steamboy,action animation drama sci fi,steampunk
702571,77629,reindeerspotting pako joulumaasta,documentary,addiction
947629,138940,duska,drama,loneliness
1072230,187613,the kissing booth,comedy romance,bff
7563,47,seven,mystery thriller,psychology


In [16]:
def tokenization_w(words):
    w_new = []
    w_token = word_tokenize(words)
    words=[word.lower() for word in w_token if word.isalpha()]
    word_token = [x for x in words if not re.fullmatch('[' + string.punctuation + ']+', x)]
    if word_token != '':
      w_new.append(word_token)
    return w_new

In [17]:
dev['title'] = dev['title'].apply(tokenization_w)
dev['genres'] = dev['genres'].apply(tokenization_w)
dev['tag'] = dev['tag'].apply(tokenization_w)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

In [18]:
dev.head()

,movieId,title,genres,tag
498480,31660,[[steamboy]],"[[action, animation, drama, sci, fi]]",[[steampunk]]
702571,77629,"[[reindeerspotting, pako, joulumaasta]]",[[documentary]],[[addiction]]
947629,138940,[[duska]],[[drama]],[[loneliness]]
1072230,187613,"[[the, kissing, booth]]","[[comedy, romance]]",[[bff]]
7563,47,[[seven]],"[[mystery, thriller]]",[[psychology]]
